In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

while 'notebooks' in os.getcwd():
    os.chdir('..')

import torch
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from sklearn.metrics import roc_auc_score

from src.models.gamma.graph_sage import GraphSAGE, LinkPredictor
from src.data.gamma.arxiv import load_data, get_val_test_edges, prepare_adjencency, get_edge_index_from_adjencency
from src.train.gamma import GammaGraphSage

In [3]:
torch.cuda.is_available()

True

In [4]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

## Data Loading

In [5]:
data = load_data()

data, val_edges, test_edges = get_val_test_edges(data, remove_from_data=True, device=device)

data = prepare_adjencency(data, to_symmetric=True)

edge_index = get_edge_index_from_adjencency(data, device)

## Training

In [9]:
gamma = GammaGraphSage(device, data.num_nodes)

In [10]:
gamma.optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.005
    maximize: False
    weight_decay: 0
)

In [14]:
predictions = gamma.predict(edge_index, data.adj_t)

In [15]:
predictions.to('cpu').detach().numpy()

array([[0.7073327 ],
       [0.7073511 ],
       [0.7073644 ],
       ...,
       [0.70729345],
       [0.7072935 ],
       [0.7073076 ]], dtype=float32)

In [17]:
gamma.is_same_class(edge_index, data.y)

tensor([[1],
        [0],
        [1],
        ...,
        [0],
        [0],
        [0]], device='cuda:0')

In [18]:
roc_auc_score(gamma.is_same_class(edge_index, data.y).to('cpu').detach().numpy(), 
              predictions.to('cpu').detach().numpy())

0.5133643827692921

In [25]:
gamma.optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.005
    maximize: False
    weight_decay: 0
)

In [103]:
embedding.weight.device

device(type='cuda', index=0)

In [67]:
data.edge_index.size()

AttributeError: 'NoneType' object has no attribute 'size'

In [40]:
val_edges.size()

torch.Size([2, 29156])

In [41]:
test_edges.size()

torch.Size([2, 29156])

In [44]:
data

Data(num_nodes=169343, edge_index=[2, 1107931], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])

In [47]:
T.ToSparseTensor()(data)

Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1107931])

In [50]:
data.adj_t = data.adj_t.to_symmetric()

In [51]:
row, col, _ = data.adj_t.coo()
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [52]:
edge_index.size()

torch.Size([2, 2200676])

In [59]:
type(data)

torch_geometric.data.data.Data

In [43]:
data.adj_t = data.adj_t.to_symmetric()

Data(num_nodes=169343, edge_index=[2, 1107931], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])

In [55]:
row, col, _ = data.adj_t.coo()